<a href="https://colab.research.google.com/github/PUNEETSUBHANJI/NYC_Taxi_Trip_Time_prediction/blob/main/Taxi_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project  : NYC Taxi trip duration Prediction :: Predicting total ride duration of taxi trips in NYC using the given Dataset

## Description of Data

The dataset is based of the 2016 NYC Cab trip record data, The data was originally published by the NYC Taxi and Limousine Commission.
The data was sampled and cleaned for the performing this project.

Based on individual trip attributes, you should predict the duration of each trip in the test set.

<b> NYC Taxi Data.csv </b> - The Dataset contains 1458644 trip records


### Details of Dataset
* #### id - A unique id for each trip.
* #### vendor_id - A code specifying the provider associated with the trip record.
* #### pickup_datetime - Pick up date and time or Time of meter started.
* #### dropoff_datetime -  Dropped off date and time or Time of meter started.
* #### passenger_count - Passenger count as entered by the driver.
* #### pickup_longitude - Longitude details of Pick up or when the meter started.
* #### pickup_latitude -  latitude details of Pick up or when the meter started.
* #### dropoff_longitude - Longitude details of dropoff or when the meter turned off.
* #### dropoff_latitude - latitude details of dropoff or when the meter turned of
* #### store_and_fwd_flag - This flag shows us, whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip.
* #### trip_duration - Its our target variable and it is duration of the trip in seconds.


# Mounting of Drive, Loading Data and Importing of the required libraries


In [6]:
# Mounting of google drive to fetch the Raw_DATA_SET
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:

# Importing necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split,GridSearchCV
import statsmodels.formula.api as sm
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import datetime as dt
%matplotlib inline
import warnings; warnings.simplefilter('ignore')

In [8]:
#Fetching the given dataet using pandas
taxi_data = pd.read_csv("/content/drive/MyDrive/Projects- Almabetter_Puneet/NYC Taxi Time prediction/NYC Taxi Data.csv")

#Details of The Data

In [9]:
#Importing first 10 indexes of the taxi_data
taxi_data.head(10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
5,id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,443
6,id1813257,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,341
7,id1324603,2,2016-05-21 07:54:58,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,1551
8,id1301050,1,2016-05-27 23:12:23,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,255
9,id0012891,2,2016-03-10 21:45:01,2016-03-10 22:05:26,1,-73.981049,40.744339,-73.973000,40.789989,N,1225


In [10]:
#Disply last 10 indexes of the dataset
taxi_data.tail(10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1458634,id3445276,1,2016-04-03 13:51:25,2016-04-03 14:07:37,2,-73.989075,40.730465,-73.963882,40.773739,N,972
1458635,id3027038,2,2016-05-19 14:46:55,2016-05-19 14:50:52,1,-73.985390,40.763020,-73.989708,40.767502,N,237
1458636,id0405770,2,2016-02-12 10:13:06,2016-02-12 10:26:26,1,-73.863815,40.769684,-73.864395,40.761326,N,800
1458637,id1920898,1,2016-04-17 18:48:16,2016-04-17 19:00:56,1,-73.975357,40.751705,-73.949478,40.776764,N,760
1458638,id1454193,2,2016-02-02 00:39:39,2016-02-02 00:46:33,5,-73.988823,40.736553,-73.989166,40.757393,N,414
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373
1458643,id1209952,1,2016-04-05 14:44:25,2016-04-05 14:47:43,1,-73.979538,40.781750,-73.972809,40.790585,N,198


In [11]:
#Total number of Rows and Columns in the given dataset
#shape of given dataframe
taxi_data.shape

(1458644, 11)

###Number of rows is:  **1458644** 
###Number of columns is:  **11**

In [12]:
# Variables present in the given dataset
taxi_data.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

In [13]:
#Datatypes of Variables present in data
taxi_data.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dtype: object

The variables such as pickup_datetime, dropoff_datetime of the type 'object'.

Converting them into type 'datetime'.

In [14]:
#  NULL/NAN values in given dataset
taxi_data.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

NULL/NAN values are not present in given dataset.

In [15]:
#  Converting them into 'datetime'
taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'])
taxi_data['dropoff_datetime'] = pd.to_datetime(taxi_data['dropoff_datetime'])

taxi_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


# Creation of Variables

In [17]:
# Creation of NEW Variables
taxi_data['pickup_day']=taxi_data['pickup_datetime'].dt.day_name()
taxi_data['dropoff_day']=taxi_data['dropoff_datetime'].dt.day_name()
taxi_data['Pickday_No']=taxi_data['pickup_datetime'].dt.weekday
taxi_data['Hour_of_Pickup']=taxi_data['pickup_datetime'].dt.hour
taxi_data['pickup_month']=taxi_data['pickup_datetime'].dt.month

Following variables have been created:

*   pickup_day : it contains the name of the day on which the 
ride was taken.
*   Pickpday_No : it contains the day number, instead of characters with Monday = 0 and Sunday = 6.

*   Hour_of_Pickup : it contains of the day in the 24 hour format.
*   pickup_month : it contains month number i`e January = 1    and December = 12.

In [18]:
#Let`s see places were booked the most using our maps
import folium
f = folium.Figure(width=1500, height=500)
mapa = folium.Map(location = (40.7679, -73.9822), zoom_start=11).add_to(f)

for index, row in taxi_data.sample(1000).iterrows():
    folium.Marker([row["pickup_latitude"], row["pickup_longitude"]], icon=folium.Icon(color="purple")).add_to(mapa)
    folium.Marker([row["dropoff_latitude"], row["dropoff_longitude"]], icon=folium.Icon(color="red")).add_to(mapa)
display(mapa)

But, we are unable to get any insights or draw conclusions from that. So,we will extract the variable from this is -> "distance".

In [23]:
# Using geographical coordinates we will calculate the distance by Importing the library
from geopy.distance import great_circle
def cal_distance(pickup_lat,pickup_long,dropoff_lat,dropoff_long):
  start_coordinates=(pickup_lat,pickup_long)
  stop_coordinates=(dropoff_lat,dropoff_long)
  return great_circle(start_coordinates,stop_coordinates).km

#    Applying the above details and creating the feature ‘distance’
taxi_data['distance'] = taxi_data.apply(lambda x: cal_distance 
                              (x['pickup_latitude'],x['pickup_longitude'],
                               x['dropoff_latitude'],x['dropoff_longitude'] ),
                               axis=1)

# Calculation of SPEED in KM per Hour
taxi_data['speed'] = (taxi_data.distance/(taxi_data.trip_duration/3600))

Create a slots of time in a day, to determine what time of the day the ride was taken.

Creating the four slots of time:-

1.  Morning ( 6:00 am to 11:59 pm),

2.  Afternoon ( 12 noon to 3:59 pm),

3.  Evening ( 4:00 pm to 9:59 pm), and

4.  Late Night ( 10:00 pm to 5:59 am)

In [24]:
# Creating the slots of time in days
def time_of_day(x):
    if x in range(6,12):
        return 'Morning'
    elif x in range(12,16):
        return 'Afternoon'
    elif x in range(16,22):
        return 'Evening'
    else:
        return 'Late night'


# Now using above function and create new columns in the dataset.
taxi_data['PickupTime_of_day']=taxi_data['Hour_of_Pickup'].apply(time_of_day)

# Dataset description after creating new variables
taxi_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,dropoff_day,Pickpday_No,Hour_of_Pickup,month,Pickday_No,pickup_month,distance,speed,Pickup_Time,PickupTime_of_day
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,Monday,0,17,3,0,3,1.498523,11.856445,Evening,Evening
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,Sunday,6,0,6,6,6,1.805510,9.803673,Late night,Late night
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,Tuesday,1,11,1,1,1,6.385108,10.822216,Morning,Morning
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,Wednesday,2,19,4,2,4,1.485501,12.465739,Evening,Evening
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,Saturday,5,13,3,5,3,1.188590,9.836608,Afternoon,Afternoon
